In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests 

sns.set()

In [ ]:
url_dead = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101G/ManadFoddDod'

json_query_dead = {
  "query": [
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    },
    {
      "code": "Manad",
      "selection": {
        "filter": "item",
        "values": [
          "01",
          "02",
          "03",
          "04",
          "05",
          "06",
          "07",
          "08",
          "09",
          "10",
          "11",
          "12",
          "90"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "000000NF"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r_dead = requests.post(url_dead,json=json_query_dead)

In [ ]:
r_dead.status_code

In [ ]:
json_dead = r_dead.json()
json_dead['data'][0]

In [ ]:
nr_rec = len(json_dead['data'])
nr_rec

In [ ]:
data_list_dead = []

for rec in range(len(json_dead['data'])):
    data_list_dead.append([json_dead['data'][rec]['key'][0],json_dead['data'][rec]['key'][1],
                    json_dead['data'][rec]['key'][2],json_dead['data'][rec]['values'][0]])
    

In [ ]:
data_list_dead

In [ ]:
df_dead = pd.DataFrame(data_list_dead,columns = ['gender','month','year','dead'])

In [ ]:
df_dead

In [ ]:
df_dead.info()

In [ ]:
df_dead['dead'] = df_dead['dead'].astype(int)
df_dead

In [ ]:
year_total_dead = df_dead.groupby(['year']).sum()
year_total_dead.index = pd.DatetimeIndex(year_total_dead.index).year
year_total_dead

In [ ]:
year_total_dead.info()

In [ ]:
year_total_dead.plot()

In [ ]:
url_pop = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101A/BefolkningR1860'

json_query_pop = {
  "query": [
    {
      "code": "Alder",
      "selection": {
        "filter": "vs:ÅlderTotA",
        "values": [
          "tot"
        ]
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r_pop = requests.post(url_pop,json = json_query_pop)
r_pop.status_code

In [ ]:
json_pop = r_pop.json()
json_pop['data'][0]

In [ ]:
data_list_pop = []

for rec in range(len(json_pop['data'])):
    data_list_pop.append([json_pop['data'][rec]['key'][1],
                         json_pop['data'][rec]['key'][2],
                         json_pop['data'][rec]['values'][0]])

In [ ]:
df_pop = pd.DataFrame(data_list_pop,columns=['gender','year','pop'])
df_pop['pop'] = df_pop['pop'].astype(int)
df_pop['year'] = df_pop['year'].astype(int)
df_pop

In [ ]:
year_total_pop = df_pop.groupby('year').sum()
year_total_pop

In [ ]:
mortality = year_total_dead['dead'] / year_total_pop['pop']
mortality.dropna(inplace=True)

In [ ]:
mortality

In [ ]:
mortality.plot()

In [ ]:
# alternative way to parse json data without explicit processing of each record

foo = pd.DataFrame.from_dict(r_pop.json()['data'])
foo

In [ ]:
pop_data = pd.DataFrame(foo['key'].to_list(),columns=['region','gender','year'])
pop_data['pop'] = foo['values'].apply(lambda x : x[0]).astype(int)

pop_data

In [ ]:
yearly_population = pop_data.groupby('year').sum()
yearly_population